In [101]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Churn_Modelling.csv')


In [13]:
rows, cols = df.shape

In [16]:
print(df.columns)

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


In [17]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
df_dropped = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [35]:
value_mapping = {'Female': '1', 'Male': '0'}


In [42]:
df_dropped["Gender"] = df_dropped["Gender"].replace(value_mapping)

In [47]:
columns = df_dropped.columns.to_list()


In [52]:
for col_name in columns:
    print(col_name + ": ", df_dropped[col_name].dtype)

CreditScore:  int64
Geography:  object
Gender:  object
Age:  int64
Tenure:  int64
Balance:  float64
NumOfProducts:  int64
HasCrCard:  int64
IsActiveMember:  int64
EstimatedSalary:  float64
Exited:  int64


In [54]:
df_dropped["Gender"] = df_dropped["Gender"].astype(int)

In [55]:
for col_name in columns:
    print(col_name + ": ", df_dropped[col_name].dtype)

CreditScore:  int64
Geography:  object
Gender:  int64
Age:  int64
Tenure:  int64
Balance:  float64
NumOfProducts:  int64
HasCrCard:  int64
IsActiveMember:  int64
EstimatedSalary:  float64
Exited:  int64


In [80]:
df_dummies = pd.get_dummies(df_dropped, columns=['Geography'], prefix=['Geography'])
columns = df_dummies.columns.to_list()
print(columns)




['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France', 'Geography_Germany', 'Geography_Spain']


In [88]:
dict_transl = {True:1, False:0}

df_dummies["Geography_France"] = df_dummies["Geography_France"].replace(dict_transl)
df_dummies["Geography_Germany"] = df_dummies["Geography_Germany"].replace(dict_transl)
df_dummies["Geography_Spain"] = df_dummies["Geography_Spain"].replace(dict_transl)


In [92]:
df_dummies.columns.to_list()

['CreditScore',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited',
 'Geography_France',
 'Geography_Germany',
 'Geography_Spain']

In [94]:
X = df_dummies[['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
        'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
        'Geography_France', 'Geography_Germany', 'Geography_Spain']]
y = df_dummies['Exited']

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [98]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [99]:
# Build a simple neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4461 - accuracy: 0.8054 - val_loss: 0.3882 - val_accuracy: 0.8325
Epoch 2/10
250/250 [==============================] - 0s 395us/step - loss: 0.3762 - accuracy: 0.8434 - val_loss: 0.3540 - val_accuracy: 0.8565
Epoch 3/10
250/250 [==============================] - 0s 390us/step - loss: 0.3516 - accuracy: 0.8551 - val_loss: 0.3476 - val_accuracy: 0.8560
Epoch 4/10
250/250 [==============================] - 0s 390us/step - loss: 0.3442 - accuracy: 0.8568 - val_loss: 0.3497 - val_accuracy: 0.8535
Epoch 5/10
250/250 [==============================] - 0s 395us/step - loss: 0.3392 - accuracy: 0.8593 - val_loss: 0.3493 - val_accuracy: 0.8580
Epoch 6/10
250/250 [==============================] - 0s 415us/step - loss: 0.3358 - accuracy: 0.8602 - val_loss: 0.3445 - val_accuracy: 0.8585
Epoch 7/10
250/250 [==============================] - 0s 406us/step - loss: 0.3330 - accuracy: 0.8627 - val_loss: 0.3420 - val_accuracy: 0

In [2]:
def make_predictions(data_array, model):
    # Create a DataFrame from the input data_array with appropriate column names
    columns = [
        'RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
        'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
        'IsActiveMember', 'EstimatedSalary'
    ]
    
    df = pd.DataFrame([data_array], columns=columns)
    
    # Perform any necessary data preprocessing to match the model's input format
    # In this case, we'll drop 'RowNumber', 'CustomerId', 'Surname' and one-hot encode 'Geography'
    df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
    df = pd.get_dummies(df, columns=['Geography'], drop_first=True)
    
    # Make the prediction
    prediction = (model.predict(df) > 0.5).astype("int32")
    
    if prediction[0] == 1:
        print("The model predicts that the customer will churn.")
    else:
        print("The model predicts that the customer will not churn.")
    